In [1]:
import requests
import lxml.html

In [38]:
root="https://www.avtogermes.ru/"
url="https://www.avtogermes.ru/sale/new/?page=2"

In [39]:
r=requests.get("http://localhost:8050/render.html",params={"url":url})

In [40]:
r.text

'<!DOCTYPE html><html prefix="og: http://ogp.me/ns/website#" itemscope="" itemtype="http://schema.org/AutoDealer" lang="ru"><head> <meta charset="utf-8"><link rel="apple-touch-icon" sizes="180x180" href="/img/favicon/apple-touch-icon.png"><link rel="icon" type="image/png" sizes="32x32" href="/img/favicon/favicon-32x32.png"><link rel="icon" type="image/png" sizes="16x16" href="/img/favicon/favicon-16x16.png"><link rel="icon" type="image/png" sizes="192x192" href="/img/favicon/android-chrome-192x192.png"><link rel="icon" type="image/png" sizes="256x256" href="/img/favicon/android-chrome-256x256.png"><link rel="manifest" href="/img/favicon/site.webmanifest"><link rel="mask-icon" href="/img/favicon/safari-pinned-tab.svg" color="#5bbad5"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="csrf-token" content="ibKezx0VZnxQpxlBz4n0i6xnUOiBln1PikddpZVq"><meta name="adv-phone" content="+7 (495) 139-52-70"><link rel

In [5]:
html=lxml.html.fromstring(r.content)

In [11]:
car_list=[root+url for url in html.xpath('//div[@class="col-6 col-md-4 col-xl-3"]/div/div[@class="buttons info"]/a/@href')]

In [1]:
def scrap_car(car_url):
    car_dict={}
    r=requests.get("http://localhost:8050/render.html",params={"url":car_url})
    html=lxml.html.fromstring(r.content)
    all_details=html.xpath('//div[@class="col-12 col-md-6"]/div[@class="row"]')
    for element in all_details:
        if element.xpath('./div[1]/text()')[0]=="Марка":
            car_dict['Brand (Марка)']=element.xpath('./div[2]/strong/text()')[0]
        if element.xpath('./div[1]/text()')[0]=="Модель":
            car_dict['Model (Модель)']=element.xpath('./div[2]/strong/text()')[0]
        if element.xpath('./div[1]/text()')[0]=="Год выпуска":
            car_dict['Year of issue (Год выпуска)']=element.xpath('./div[2]/strong/text()')[0]
        if element.xpath('./div[1]/text()')[0]=="Цена с учетом выгоды":
            car_dict['Price with benefits (Цена с учетом выгоды)']=element.xpath('./div[2]/strong/text()')[0]
        if element.xpath('./div[1]/text()')[0]=="Кузов":
            car_dict['Body (Цена с учетом выгоды)']=element.xpath('./div[2]/strong/text()')[0]
    return car_dict

In [4]:
def scrap_page_car(page_number,status):
    root="https://www.avtogermes.ru/"
    url=f"https://www.avtogermes.ru/sale/{status}/?page={page_number}"
    r=requests.get("http://localhost:8050/render.html",params={"url":url})
    html=lxml.html.fromstring(r.content)
    brand_model=html.xpath('//div[@class="shaded h3"]/text()[1]')
    no_discount_price_list=list()
    no_discount_price=html.xpath('//div[@class="price"]')
    for element in no_discount_price:
        if len(element.xpath('./b'))>0:
            no_discount_price_list.append(element.xpath('./b/text()')[0])
        if len(element.xpath('./strong'))>0:
            no_discount_price_list.append(element.xpath('./strong/text()')[0])
    #iscount_price=html.xpath('//div[@class="price"]/b')
    year=html.xpath('//div[@class="p dop-info shaded"]/strong[1]/text()')
    #print(len(brand_model))
    #print(len(no_discount_price_list))
    #rint(len(discount_price))
    #print(len(year))
    assert len(brand_model)==len(no_discount_price_list)==len(year)
    return {"brand_model":brand_model,
            "price":no_discount_price_list,
            "year":year}

In [12]:
test=scrap_page_car(134,"new")

In [13]:
test

{'brand_model': ['LADA Largus фургон NEW',
  'LADA Largus фургон NEW',
  'LADA Granta',
  'LADA Largus универсал NEW',
  'LADA Largus универсал NEW',
  'LADA Largus универсал NEW',
  'LADA Niva Legend 3-дв.',
  'LADA Largus Cross NEW',
  'LADA Niva Legend Urban 3-дв.',
  'LADA Vesta SW Cross',
  'CHERY Tiggo 4',
  'Cheryexeed TXL',
  'RENAULT Duster NEW',
  'Kia Carnival',
  'Kia Rio NEW',
  'Kia Rio NEW',
  'Kia Rio NEW',
  'Kia Rio NEW',
  'Kia Cerato',
  'Kia Cerato',
  'Kia K5',
  'LADA Granta Cross',
  'LADA Largus фургон NEW',
  'Kia Sorento NEW'],
 'price': ['779 900\xa0₽',
  '779 900\xa0₽',
  '539 500\xa0₽',
  '731 900\xa0₽',
  '731 900\xa0₽',
  '743 900\xa0₽',
  '598 900\xa0₽',
  '930 900\xa0₽',
  '657 900\xa0₽',
  '1 064 900\xa0₽',
  '1 269 900\xa0₽',
  '2 249 900\xa0₽',
  '1 235 000\xa0₽',
  '3 629 900\xa0₽',
  '1 009 900\xa0₽',
  '1 009 900\xa0₽',
  '1 034 900\xa0₽',
  '1 034 900\xa0₽',
  '1 385 900\xa0₽',
  '1 385 900\xa0₽',
  '2 132 900\xa0₽',
  '688 900\xa0₽',
  '779 900

In [30]:
new_cars=[]
for url_no in range(1,156):
    try:
        cars=scrap_page_car(url_no,"new")
        if len(cars['year'])>0:
            new_cars.append(cars)
        else:
            print(url_no)
    except:
        print(url_no)
        continue

16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155


In [18]:
import pandas as pd
import ftfy
def create_fix_df(car_dict):
    df=pd.DataFrame(car_dict)
    df['brand']=df.brand_model.apply(lambda x: x.split()[0])
    df['model']=df.brand_model.apply(lambda x: " ".join(x.split()[1:]))
    df["price"]=df.price.apply(lambda x: ftfy.fix_encoding(x))
    return df[["brand","model","year","price"]]

In [22]:
test=[len(create_fix_df(car_dict)) for car_dict in new_cars ]

In [29]:
[car_dict for car_dict in new_cars if len(car_dict['year'])==0]

[{'brand_model': [], 'price': [], 'year': []},
 {'brand_model': [], 'price': [], 'year': []},
 {'brand_model': [], 'price': [], 'year': []},
 {'brand_model': [], 'price': [], 'year': []}]

In [21]:
pd.concat([create_fix_df(car_dict) for car_dict in new_cars if len(car_dict['year'])>0],axis=0)

,brand,model,year,price
0,RENAULT,Duster NEW,2021,1 064 000 ₽
1,LADA,Granta хэтчбек,2021,465 900 ₽
2,SUZUKI,SX4,2021,1 459 990 ₽
3,Kia,Rio NEW,2021,1 220 900 ₽
4,УАЗ,Пикап New,2021,1 172 000 ₽
...,...,...,...,...
7,Kia,Rio NEW,2021,1 009 900 ₽
8,Kia,Rio NEW,2021,1 009 900 ₽
9,Kia,Sportage,2021,1 972 900 ₽
10,HYUNDAI,Solaris NEW,2021,871 000 ₽


,brand,model,year,price
0,RENAULT,Duster NEW,2021,1 064 000 ₽
1,LADA,Granta хэтчбек,2021,465 900 ₽
2,SUZUKI,SX4,2021,1 459 990 ₽
3,Kia,Rio NEW,2021,1 220 900 ₽
4,УАЗ,Пикап New,2021,1 172 000 ₽
5,SUZUKI,Jimny,2020,1 479 000 ₽
6,УАЗ,Патриот New,2021,1 359 900 ₽
7,УАЗ,Патриот New,2021,858 000 ₽
8,CHERY,Tiggo 7 Pro,2020,1 339 900 ₽
9,HYUNDAI,Solaris NEW,2021,916 000 ₽


In [9]:
# Cell is created mainly to catch errors in case Docker shuts down suddenly. Adjust to your own usage.
for url_no in range(134,156):
    try:
        cars=scrap_page_car(url_no,"new")
        new_cars.extend(cars)
    except:
        print(url_no)
        continue

In [15]:
new_cars

[{'brand_model': ['RENAULT Duster NEW',
   'LADA Granta хэтчбек',
   'SUZUKI SX4',
   'Kia Rio NEW',
   'УАЗ Пикап New',
   'SUZUKI Jimny',
   'УАЗ Патриот New',
   'УАЗ Патриот New',
   'CHERY Tiggo 7 Pro',
   'HYUNDAI Solaris NEW',
   'CITROEN C3 Aircross',
   'RENAULT Duster NEW',
   'CITROEN C5 Aircross',
   'RENAULT Kaptur',
   'Kia K5',
   'RENAULT Sandero',
   'HYUNDAI Elantra',
   'SUZUKI Jimny',
   'Kia Sorento NEW',
   'УАЗ Патриот New',
   'УАЗ Патриот New',
   'MITSUBISHI ASX NEW',
   'SUZUKI SX4',
   'Kia Ceed'],
  'price': ['1 064 000\xa0₽',
   '465 900\xa0₽',
   '1 459 990\xa0₽',
   '1 220 900\xa0₽',
   '1 172 000\xa0₽',
   '1 479 000\xa0₽',
   '1 359 900\xa0₽',
   '858 000\xa0₽',
   '1 339 900\xa0₽',
   '916 000\xa0₽',
   '1 564 000\xa0₽',
   '969 000\xa0₽',
   '2 514 000\xa0₽',
   '1 457 000\xa0₽',
   '2 022 900\xa0₽',
   '778 000\xa0₽',
   '1 394 000\xa0₽',
   '1 479 000\xa0₽',
   '2 342 900\xa0₽',
   '883 900\xa0₽',
   '1 304 900\xa0₽',
   '1 312 000\xa0₽',
   '1 439

In [47]:
def scrape_page_car_final(page_number,status):
    try:
        item=scrap_page_car(page_number,status)
        return item
    except:
        return None

In [43]:
new_cars=[scrape_page_car_final(url_no,"new") for url_no in range(1,156)]

In [48]:
testing=scrape_page_car_final(1,"new")

In [52]:
print(scrap_page_car(1,"new"))

[{'Brand (Марка)': 'RENAULT', 'Model (Модель)': 'Duster NEW', 'Year of issue (Год выпуска)': '2021', 'Body (Цена с учетом выгоды)': 'Внедорожник', 'Price with benefits (Цена с учетом выгоды)': '1 064 000 руб.'}, {'Brand (Марка)': 'LADA', 'Model (Модель)': 'Granta хэтчбек', 'Year of issue (Год выпуска)': '2021', 'Body (Цена с учетом выгоды)': 'Хэтчбек', 'Price with benefits (Цена с учетом выгоды)': '465 900 руб.'}, {'Brand (Марка)': 'SUZUKI', 'Model (Модель)': 'SX4', 'Year of issue (Год выпуска)': '2021', 'Body (Цена с учетом выгоды)': 'Кроссовер', 'Price with benefits (Цена с учетом выгоды)': '1 459 990 руб.'}, {'Brand (Марка)': 'Kia', 'Model (Модель)': 'Rio NEW', 'Year of issue (Год выпуска)': '2021', 'Body (Цена с учетом выгоды)': 'Седан', 'Price with benefits (Цена с учетом выгоды)': '1 220 900 руб.'}, {'Brand (Марка)': 'УАЗ', 'Model (Модель)': 'Пикап New', 'Year of issue (Год выпуска)': '2021', 'Body (Цена с учетом выгоды)': 'Внедорожник', 'Price with benefits (Цена с учетом выгоды

In [ ]:
old_cars=[]
for url_no_2 in range(1,62):
    old_cars.extend(scrap_page_car(url_no_2,"second_hand"))

In [29]:
new_cars

[{},
 {'Brand (Марка)': 'Kia',
  'Model (Модель)': 'Ceed SW',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Универсал',
  'Price with benefits (Цена с учетом выгоды)': '1 524 900 руб.'},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {}]

[{'Brand (Марка)': 'RENAULT',
  'Model (Модель)': 'Duster NEW',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Внедорожник',
  'Price with benefits (Цена с учетом выгоды)': '1 064 000 руб.'},
 {'Brand (Марка)': 'LADA',
  'Model (Модель)': 'Granta хэтчбек',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Хэтчбек',
  'Price with benefits (Цена с учетом выгоды)': '465 900 руб.'},
 {'Brand (Марка)': 'SUZUKI',
  'Model (Модель)': 'SX4',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Кроссовер',
  'Price with benefits (Цена с учетом выгоды)': '1 459 990 руб.'},
 {'Brand (Марка)': 'Kia',
  'Model (Модель)': 'Rio NEW',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Седан',
  'Price with benefits (Цена с учетом выгоды)': '1 220 900 руб.'},
 {'Brand (Марка)': 'УАЗ',
  'Model (Модель)': 'Пикап New',
  'Year of issue (Год выпуска)': '2021',
  'Body (Цена с учетом выгоды)': 'Внедорожник',
